This notebook demonstrates the procedure presented in

F. Picetti, F. Lombardi, P. Bestagini, M. Lualdi, S. Tubaro. <i>Landmine Detection Using Autoencoders on Multi-polarization GPR Volumetric Data</i>, submitted to <i>IEEE Transactions on Geoscience and Remote Sensing (TGRS)</i>, Sept. 2018

In [11]:
import os
import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

import PreProcessing
import net
from python_patch_extractor import PatchExtractor

from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# Variables

In [12]:
in_path = 'giuriati_2/20170621_deg0_HHVV.npy'
out_path = 'cnn_article'
architecture = 'Auto3D2'
ny = 3 # number of adjacent B-scans to be considered
data_augmentation = True
preprocessing = 'normalize'
patch_size = 64
patch_stride = 4
n_bsc = 5 # number of B_scans for training

In [13]:
def parse_pp(string):
    return getattr(PreProcessing, string)

def parse_net(string):
    return getattr(net, string)

In [14]:
if not os.path.exists(out_path):
    os.makedirs(out_path)

field, campaign = in_path.split('/')
campaign, extension = campaign.split('.')

# Load Datasets

In [15]:
dataset = np.load('./datasets/'+str(in_path)).item()

train_bsc_idx = np.where(np.asarray(dataset['ground_truth']) == 0)[0][:n_bsc]
trainset = dataset['data'][train_bsc_idx]
trainset = np.moveaxis(trainset, np.argmin(trainset.shape), -1)
del dataset

### block extraction

In [16]:
if patch_size is not None:
    patch_size = (patch_size, patch_size)
else:
    patch_size = trainset.shape[1:]

patch_size = patch_size + (ny,)
patch_stride = (patch_stride, patch_stride, 1)

pe = PatchExtractor.PatchExtractor(patch_size, stride=patch_stride)

train_patches = pe.extract(trainset)
# reshaping
train_patches = train_patches.reshape((-1,) + patch_size)

# preprocessing each patch
train_patches, min_tr, max_tr = PreProcessing.apply_transform(train_patches, transform=parse_pp(preprocessing))

# Data augmentation (default=True)
if data_augmentation:
    train_patches = np.concatenate([train_patches, np.flip(train_patches, axis=2).copy()], axis=0)

train_patches = shuffle(train_patches)

# create training and validation sets
train_patches, val_patches, train_index, val_index = train_test_split(train_patches,
                                                                      np.arange(train_patches.shape[0]),
                                                                      test_size=0.5,
                                                                      random_state=118
                                                                      )

# CNN Architecture

In [17]:
sets = net.Settings()
patience = sets.patience
lr_factor = sets.lr_factor
batch_size = sets.batch_size
epochs = sets.epochs

autoencoder, encoder = parse_net(architecture)(patch_size)

out_name = field+'_'+campaign+'_'+architecture+'_patch'+str(patch_size[0])+'_stride'+str(patch_stride[0])+'_bsc'+str(n_bsc)+'_ny'+str(ny)

lr_chkpt = ReduceLROnPlateau(monitor='val_loss',
                             factor=lr_factor,
                             patience=patience//2,
                             verbose=0,
                             mode='auto',
                             epsilon=0.0001,
                             cooldown=0,
                             min_lr=0)
save_chkpt = ModelCheckpoint(os.path.join(out_path, out_name+'.h5'),
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True,
                             save_weights_only=True,
                             mode='min')
stop_chkpt = EarlyStopping(monitor='val_loss',
                           patience=patience)

/opt/miniconda3/lib/python3.6/site-packages/keras/callbacks.py:999: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


In [18]:
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
g_in_0 (InputLayer)          (None, 64, 64, 3)         0         
_________________________________________________________________
g_conv_0 (Conv2D)            (None, 64, 64, 16)        1744      
_________________________________________________________________
g_conv_1 (Conv2D)            (None, 32, 32, 16)        6416      
_________________________________________________________________
g_conv_2 (Conv2D)            (None, 16, 16, 16)        4112      
_________________________________________________________________
g_conv_3 (Conv2D)            (None, 8, 8, 16)          2320      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 4, 4, 16)          1040      
_________________________________________________________________
encoder (Conv2D)             (None, 2, 2, 16)          272       
__________

### Training

In [8]:
train = autoencoder.fit(train_patches, train_patches,
                        validation_data=(val_patches, val_patches),
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=0,
                        callbacks=[save_chkpt, stop_chkpt, lr_chkpt])
print('Training done!')


Epoch 00001: val_loss improved from inf to 0.02289, saving model to cnn_article/giuriati_2_20170621_deg0_HHVV_Auto3D2_patch64_stride4_bsc5_ny3.h5

Epoch 00002: val_loss improved from 0.02289 to 0.01759, saving model to cnn_article/giuriati_2_20170621_deg0_HHVV_Auto3D2_patch64_stride4_bsc5_ny3.h5

Epoch 00003: val_loss improved from 0.01759 to 0.01580, saving model to cnn_article/giuriati_2_20170621_deg0_HHVV_Auto3D2_patch64_stride4_bsc5_ny3.h5

Epoch 00004: val_loss improved from 0.01580 to 0.01499, saving model to cnn_article/giuriati_2_20170621_deg0_HHVV_Auto3D2_patch64_stride4_bsc5_ny3.h5

Epoch 00005: val_loss improved from 0.01499 to 0.01433, saving model to cnn_article/giuriati_2_20170621_deg0_HHVV_Auto3D2_patch64_stride4_bsc5_ny3.h5

Epoch 00006: val_loss improved from 0.01433 to 0.01394, saving model to cnn_article/giuriati_2_20170621_deg0_HHVV_Auto3D2_patch64_stride4_bsc5_ny3.h5

Epoch 00007: val_loss improved from 0.01394 to 0.01349, saving model to cnn_article/giuriati_2_20

# Deployment (test)

In [17]:
from sklearn.metrics import roc_curve, roc_auc_score
from tqdm import tqdm

In [18]:
training = './cnn_article/giuriati_2_20170621_deg0_HHVV_Auto3D2_patch64_stride4_bsc5_ny3'
net_weights = training + '.h5'

### Load dataset

In [19]:
# in this case, the dataset for training in the same for testing
train_path = in_path
dataset = np.load('./datasets/' + str(in_path)).item()

# preprocessing
data = dataset['data']

# patch extractor
pe = PatchExtractor.PatchExtractor(patch_size, stride=patch_stride)

# background bscans for training
gt = np.asarray(dataset['ground_truth'])
del dataset

test_idx = np.arange(data.shape[0])
# check whether the test dataset is the same of the training
if in_path == train_path:
    train_idx = np.where(gt == 0)[0][:n_bsc]
    test_idx = np.delete(test_idx, train_idx)
testset = data[test_idx]
gt = gt[test_idx]
del data
testset = np.moveaxis(testset, np.argmin(testset.shape), -1)

### Load architecture

In [20]:
autoencoder, encoder = parse_net(architecture)(patch_size)
autoencoder.load_weights(os.path.join(net_weights))

out_name = field+'_'+campaign+'_'+architecture+'_patch'+str(patch_size[0])+'_stride'+str(patch_stride[0])+'_bsc'+str(n_bsc)+'_ny'+str(ny)

### Test loop

In [21]:
patches = pe.extract(testset)
del testset
patchesIdx = patches.shape
patches_hat = autoencoder.predict(patches.reshape((-1,) + patch_size))
mseFeat = (encoder.predict(patches.reshape((-1,) + patch_size)) - encoder.predict(patches_hat))**2
mseFeat_patches = np.zeros(patches_hat.shape) + np.mean(mseFeat, axis=(1,2,3)).reshape((-1,1,1,1))
del patches
del patches_hat
del mseFeat
mseFeat_vol = pe.reconstruct(mseFeat_patches.reshape(patchesIdx))
del mseFeat_patches

### Evaluation

In [22]:
mse_mask_max = np.max(mseFeat_vol, axis=(0, 1))
fpr_max, tpr_max, thresholds_max = roc_curve(gt, mse_mask_max)
roc_auc_max = roc_auc_score(gt, mse_mask_max)
print('best AUC = %0.2f' % roc_auc_max)

best AUC = 0.93
